In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium 
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
from glob import glob
import os
pbar = ProgressBar()
pbar.register()

from tqdm import tqdm_notebook as tqdm
import geopandas as gpd

### 좌표변환

In [ ]:
from pyproj import Proj, transform
proj_EPSG_5186 = Proj(init = 'epsg:5186')
# proj_WGS84 = Proj(init = 'epsg:4326')
proj_EPSG_5179 = Proj(init = 'epsg:5179')

def transform_axis(df):
    return pd.Series(transform(proj_EPSG_5186, proj_EPSG_5179,df['X_AXIS'],df['Y_AXIS']),index = ['X','Y'])


### data 통합후 좌표변환

In [ ]:
file_paths = glob('E:/python/data/KT 데이터/wlk/*.csv')

yearly_2016 = pd.DataFrame()
yearly_2017 = pd.DataFrame()
for i in tqdm(range(len(file_paths))):
    year = file_paths[i].split('_')[1][:4]
    sample = pd.read_csv(file_paths[i])
    if year == '2016':
        yearly_2016 = pd.concat([yearly_2016,sample])
    if year == '2017':
        yearly_2017 = pd.concat([yearly_2017,sample])

In [ ]:
axis_2016 = yearly_2016[['X_AXIS','Y_AXIS']].drop_duplicates()
axis_2017 = yearly_2017[['X_AXIS','Y_AXIS']].drop_duplicates()

In [ ]:
axis_all = pd.concat([axis_2016,axis_2017])
axis_all = axis_all.drop_duplicates()

In [ ]:
%%time
#좌표변환
axis_all[['new_X','new_y']] = axis_all.apply(transform_axis,axis=1)

In [ ]:
nw_2016 = pd.merge(yearly_2016,axis_all,on = ['X_AXIS','Y_AXIS'], how = 'left')
nw_2017 = pd.merge(yearly_2017,axis_all,on = ['X_AXIS','Y_AXIS'], how = 'left')

In [ ]:
yearly_2016.shape, nw_2016.shape, yearly_2017.shape, nw_2017.shape

In [ ]:
nw_2016.drop(['X_AXIS','Y_AXIS'],axis = 1, inplace = True)
nw_2017.drop(['X_AXIS','Y_AXIS'],axis = 1, inplace = True)

In [ ]:
nw_2016.to_csv('./data/KT 데이터/2016_wlk_통합(5179).csv',index= False)
nw_2017.to_csv('./data/KT 데이터/2017_wlk_통합(5179).csv',index= False)